In [1]:
from embedding_search.vector_store import MiniStore

store = MiniStore()
store.build()

In [2]:
y = store.search("pandemic resilience")

In [3]:
y

[{'type': 'article',
  'orcid_path': '/0000-0001-7079-4473/work/111834895',
  'doi': '10.1016/j.seps.2021.101174',
  'title': 'Designing pandemic-resilient voting systems',
  'url': 'http://www.scopus.com/inward/record.url?eid=2-s2.0-85118349597&partnerID=MN8TOARS',
  'publication_year': 2022,
  'pulled_abstract': True,
  'raw_abstract': None,
  'abstract': None,
  'distance': 0.13028320841527086},
 {'type': 'article',
  'orcid_path': '/0000-0001-7079-4473/work/111834893',
  'doi': '10.1080/23789689.2017.1294859',
  'title': 'Defining resilience analytics for interdependent cyber-physical-social networks',
  'url': 'http://www.scopus.com/inward/record.url?eid=2-s2.0-85082053489&partnerID=MN8TOARS',
  'publication_year': 2017,
  'pulled_abstract': True,
  'raw_abstract': None,
  'abstract': None,
  'distance': 0.15416612820841402},
 {'type': 'article',
  'orcid_path': '/0000-0001-7079-4473/work/57988968',
  'doi': '10.1016/j.ress.2018.12.006',
  'title': 'Resilience-driven restoration m

# Network graph

In [ ]:
!pip install matplotlib

In [ ]:
import altair as alt
import pandas as pd

# Create a DataFrame with the two points
data = pd.DataFrame({"x": [0, 1], "y": [0, 0.5]})

# Create the Altair chart with a line connecting the two points
chart = alt.Chart(data).mark_line().encode(x="x", y="y")

# Display the chart
chart

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create an empty graph
G = nx.Graph()

# Add author nodes
G.add_nodes_from([1, 2, 3, 4, 5])
node_colors = ["lightblue" for _ in range(5)]

# Add article nodes
G.add_nodes_from([6, 7, 8])
node_colors.extend(["red" for _ in range(3)])

# Add edges
G.add_edges_from([(1, 6), (1, 7), (2, 8), (3, 6)])

# Plot the graph


nx.draw(G, with_labels=True, node_color=node_colors, edge_color="gray")

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import altair as alt

nodes = ['a', 'b', 'c']
edges = 

df_nodes = pd.DataFrame(nodes, columns=["id"])
df_edges = pd.DataFrame(edges, columns=["source", "target"])
    
# Create an Altair chart
chart = alt.Chart(df_edges).mark_circle(size=200).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None),
    color=alt.Color('source:N', legend=None)
)

# Add tooltips to the chart
chart = chart.encode(
    tooltip=['source', 'target']
)

# Add force layout transform to the chart
chart = chart.transform_lookup(
    lookup='id',
    from_=alt.LookupData(data=df_nodes, key='id', fields=['x', 'y'])
).transform_force(
    force='center',
    x='x',
    y='y'
)

# Set the chart size and remove axis
chart = chart.properties(width=600, height=400).configure_axis(
    grid=False,
    ticks=False,
    labels=False
)

chart